In [1]:
import comparer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", 100)
pd.set_option('display.max_columns', None)

In [3]:
import os
os.chdir("../")
os.getcwd()

'/Users/shorouqza/Code/Wikimpacts'

In [3]:
#dev_set_filepath = "Database/output/nlp4climate/dev_set.parquet"
dev_set_filepath = "Database/output/nlp4climate/mixtral-8x7B.parquet"
dev = pd.read_parquet(dev_set_filepath, engine="pyarrow").replace({np.nan: None, "NULL ": None, "NULL": None})

dev_gold_filepath = "Loose_Excel_Sheets/Ni_282_events_Shorouq_0510.xlsx"
gold =  pd.read_excel(dev_gold_filepath, sheet_name="validation_manual_check", engine="openpyxl").replace({np.nan: None, "NULL ": None, "NULL": None})

In [4]:
print(len(dev.sort_values("Event_ID")["Event_ID"].to_list()))
print(len(gold.sort_values("Event_ID")["Event_ID"].to_list()))

100
100


In [5]:
set(dev.sort_values("Event_ID")["Event_ID"].to_list()) ^ set(gold.sort_values("Event_ID")["Event_ID"].to_list())

# TODO: these two events had no valid date and thus had been removed
# don't drop anything from the script and rerun for these two events

set()

In [6]:
# for now, drop the two missing events
# gold = gold[gold["Event_ID"] != "6FnYxig"]
# gold = gold[gold["Event_ID"] != "euvNjqT"]

In [7]:
gold[gold["Event_ID"] == "euvNjqT"]

,Unnamed: 0,note,Event_ID,Event_Name,Main_Event,Source,Event_Name.1,Start_Date_Year,Start_Date_Month,Start_Date_Day,End_Date_Year,End_Date_Month,End_Date_Day,Num_Injured,Num_Injured_Min,Num_Injured_Max,Total_Deaths,Total_Deaths_Min,Total_Deaths_Max,Country_Norm,Countries,Country,Location,Country_Extracted,Locations_Extracted,Country_Norm.1,Country_Type,Country_GeoJson,Country_GID,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
34,19,None,euvNjqT,2010-2011 China Drought,Drought,https://en.wikipedia.org/wiki?curid=30855389,2010-2011 china drought,2010,9,1,2010,9,30,None,None,None,None,None,None,"['Anhui', 'Gansu', 'Henan', 'Hubei', 'Jiangsu', 'Shaanxi', 'Shandong', 'Shanxi']",Anhui|Gansu|Henan|Hubei|Jiangsu|Shaanxi|Shandong|Shanxi,"['Anhui', 'Gansu', 'Henan', 'Hubei', 'Jiangsu', 'Shaanxi', 'Shandong', 'Shanxi']","[[], [], [], [], [], [], [], []]","['Anhui', 'Gansu', 'Henan', 'Hubei', 'Jiangsu', 'Shaanxi', 'Shandong', 'Shanxi']","[[], [], [], [], [], [], [], []]","['Anhui', 'Gansu', 'Henan', 'Hubei', 'Jiangsu', 'Shaanxi', 'Shandong', 'Shanxi']","['administrative', 'administrative', 'administrative', 'administrative', 'administrative', 'admi...","['{""type"": ""MultiPolygon"", ""coordinates"": [[[[114.8761629, 32.9960464], [114.8764352, 32.9960296...","[['CHN.1_1'], ['CHN.5_1'], ['CHN.12_1'], ['CHN.13_1'], ['CHN.15_1'], ['CHN.22_1'], ['CHN.23_1'],...",None,None,None,None,None


In [8]:
dev[dev["Event_ID"] == "euvNjqT"]

,Total_Deaths,Total_Death_Annotation,Event_Name,Main_Event,Main_Event_Assessment_With_Annotation,Start_Date,End_Date,Time_with_Annotation,Country_Merge,Countries_Affected,Country_with_Annotation,Specific_Instance_Per_Country_Death,Event_ID,Start_Date_Day,Start_Date_Month,Start_Date_Year,End_Date_Day,End_Date_Month,End_Date_Year,Total_Deaths_Min,Total_Deaths_Max,Total_Deaths_Approx,Country_Norm,Country_Type,Country_GeoJson,Country_GID
9,None,The article does not provide information on the total number of deaths directly or indirectly ca...,2010-2011_China_Drought,Drought,The 2010-2011 China drought was a drought that began in late 2010 and impacted eight provinces i...,None,None,None,[],None,"{'Anhui': 'The provinces of Anhui, Gansu, Henan, Hubei, Jiangsu, Shaanxi, Shandong, and Shanxi w...",[],euvNjqT,None,None,None,None,None,None,None,None,None,[],[],[],None


In [9]:
set(dev.sort_values("Event_ID")["Event_ID"].to_list()) ^ set(gold.sort_values("Event_ID")["Event_ID"].to_list())

set()

In [10]:
# Specify null penalty
null_penalty = 1
# Specify item weights
weights = {
        "Event_ID": 1,
        "Main_Event": 1,
        "Event_Name": 0,
        "Total_Deaths_Min": 1,
        "Total_Deaths_Max": 1,
        #"Total_Damage_Min": 1,
        #"Total_Damage_Max": 1,
        #"Total_Damage_Units": 1,
        #"Total_Damage_Inflation_Adjusted": 1,
        #"Total_Damage_Inflation_Adjusted_Year": 1,
        "Start_Date_Day": 1,
        "Start_Date_Month": 1,
        "Start_Date_Year": 1,
        "End_Date_Day": 1,
        "End_Date_Month": 1,
        "End_Date_Year": 1,
        "Country_Norm": 1,
         
         }


# Instantiate comparer
comp = comparer.Comparer(null_penalty, target_columns=list(weights.keys()))

In [11]:
comp.target_columns

['Event_ID',
 'Main_Event',
 'Event_Name',
 'Total_Deaths_Min',
 'Total_Deaths_Max',
 'Start_Date_Day',
 'Start_Date_Month',
 'Start_Date_Year',
 'End_Date_Day',
 'End_Date_Month',
 'End_Date_Year',
 'Country_Norm']

In [12]:
dev = dev.sort_values("Event_ID")
gold = gold.sort_values("Event_ID")


In [13]:
for i in  ["Total_Damage_Min",
        "Total_Damage_Max",
        "Total_Damage_Units",
        "Total_Damage_Inflation_Adjusted",
        "Total_Damage_Inflation_Adjusted_Year"]:

    gold[i] = None

In [14]:
dev["Country_Norm"] = dev["Country_Norm"].apply(ast.literal_eval)

In [15]:
gold["Country_Norm"] = gold["Country_Norm"].apply(ast.literal_eval)

In [17]:
dev_data = dev[list(weights.keys())].to_dict(orient="records")
gold_data = gold[list(weights.keys())].to_dict(orient="records")

In [18]:
dev_data[98]

{'Event_ID': 'zENLuHy',
 'Main_Event': 'Extreme Temperature; Drought',
 'Event_Name': '2022_Dallas_floods',
 'Total_Deaths_Min': 1.0,
 'Total_Deaths_Max': 1.0,
 'Start_Date_Day': None,
 'Start_Date_Month': None,
 'Start_Date_Year': None,
 'End_Date_Day': None,
 'End_Date_Month': None,
 'End_Date_Year': None,
 'Country_Norm': ['United States']}

In [19]:
gold_data[98]

{'Event_ID': 'zENLuHy',
 'Main_Event': 'Flood',
 'Event_Name': 'July–August 2022 United States floods',
 'Total_Deaths_Min': 1,
 'Total_Deaths_Max': 1,
 'Start_Date_Day': 21,
 'Start_Date_Month': 8,
 'Start_Date_Year': 2022,
 'End_Date_Day': 22,
 'End_Date_Month': 8,
 'End_Date_Year': 2022,
 'Country_Norm': ['United States']}

In [20]:
pairs = zip(dev_data, gold_data)

In [21]:
# Precision and recall (in order)
comp.events(gold_data, dev_data, weights)

(0.45088808285938436, 0.546841249968102)

In [22]:
comps = [[dev["Event_ID"], gold["Event_ID"], comp.weighted(dev, gold, weights), comp.all(dev, gold)] for (dev, gold) in pairs]

In [23]:
comps

[['2RpfEqC',
  '2RpfEqC',
  (0.0, 0.28787878787878785),
  {'Event_ID': 0,
   'Event_Name': 1,
   'Main_Event': 0,
   'Country_Norm': 1.0,
   'End_Date_Year': 0.0,
   'Total_Deaths_Max': 1,
   'Start_Date_Month': 0.0,
   'Start_Date_Year': 0.0,
   'End_Date_Day': 0.16666666666666666,
   'Total_Deaths_Min': 1,
   'End_Date_Month': 0.0,
   'Start_Date_Day': 0.0}],
 ['3YHqjVd',
  '3YHqjVd',
  (0.0, 0.13636363636363635),
  {'Event_ID': 0,
   'Event_Name': 1,
   'Main_Event': 1,
   'Country_Norm': 0.5,
   'End_Date_Year': 0.0,
   'Total_Deaths_Max': 0.0,
   'Start_Date_Month': 0.0,
   'Start_Date_Year': 0.0,
   'End_Date_Day': 0.0,
   'Total_Deaths_Min': 0.0,
   'End_Date_Month': 0.0,
   'Start_Date_Day': 0.0}],
 ['4eNzHf6',
  '4eNzHf6',
  (0.0, 0.8051948051948052),
  {'Event_ID': 0,
   'Event_Name': 1,
   'Main_Event': 0,
   'Country_Norm': 0.8571428571428572,
   'End_Date_Year': 1,
   'Total_Deaths_Max': 1,
   'Start_Date_Month': 1,
   'Start_Date_Year': 1,
   'End_Date_Day': 1,
   'Total_

In [24]:
all_comps = pd.DataFrame([[i, j, c, d] + list(a.values()) for [i, j, (c, d), a] in comps], 
                         columns=["Event_ID1", "Event_ID2", "Coverage", "Weighted_Score"] + 
                         list(weights.keys())).replace({np.nan: None})

In [25]:
all_comps.sort_values("Weighted_Score")

,Event_ID1,Event_ID2,Coverage,Weighted_Score,Event_ID,Main_Event,Event_Name,Total_Deaths_Min,Total_Deaths_Max,Start_Date_Day,Start_Date_Month,Start_Date_Year,End_Date_Day,End_Date_Month,End_Date_Year,Country_Norm
83,sk4QAb5,sk4QAb5,0.0,0.0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,8BwkFwh,8BwkFwh,0.0,0.0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,Uue2wuQ,Uue2wuQ,0.0,0.0,0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,QGcSeGr,QGcSeGr,0.0,0.036364,0,1,0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,uf4xkSS,uf4xkSS,0.0,0.048644,0,1,0,0.5,0.0,0.017544,0.0,0.0,0.0,0.017544,0.0,0.0
60,dCBLBht,dCBLBht,0.0,0.060193,0,1,0,0.230769,0.0,0.431353,0.0,0.0,0.0,0.0,0.0,0.0
44,USmZnAm,USmZnAm,0.0,0.080808,0,1,0,0.888889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,C4J4mo6,C4J4mo6,0.0,0.090909,0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57,bNFMGBA,bNFMGBA,0.0,0.090909,0,1,0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2
69,gbK82nx,gbK82nx,0.0,0.090909,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
all_comps.to_csv("Database/output/nlp4climate/mixtral-8x7B_results.csv", index=False)